In [2]:
import numpy as np
import cvxpy as cp

In [21]:
cp.atoms.norm(As[0] - A_mean, ord=2, axis=(1,2))

ValueError: Expressions of dimension greater than 2 are not supported.

In [24]:
N = 10
As = np.array([np.random.random((3, 3)) for _ in range(N)])
A_mean = cp.Variable(As[0].shape)
obj = sum([cp.atoms.norm(A - A_mean, 2) for A in As])
prob = cp.Problem(cp.Minimize(obj))
prob.solve()

6.334322292488183

In [19]:
A_hat = np.random.random((3, 3))
diff = 
diff_norm = np.linalg.
np.sum(diff_norm)

8.646932987007784

In [12]:
diff[3]

array([[ 0.53841444, -0.06177909,  0.60321197],
       [ 0.68682564, -0.54132867, -0.12289974],
       [ 0.14950711, -0.34937422,  0.21026322]])

In [1]:
import os
import pickle
import matplotlib.pyplot as plt
import numpy as np
import torch

from train_dynamics import generate_data
from policygradient import PolicyGradientOptions, run_policy_gradient, run_dynamics_gradient, Regularizer
from ltimult import LQRSysMult

In [2]:
device = "cpu"
cached_fn = os.path.join("dynamics_trained", "dynamics.nf")
with open(cached_fn, "rb") as f:
    encoder = pickle.load(f)
encoder.to(device)

Flow(
  (_transform): CompositeTransform(
    (_transforms): ModuleList(
      (0): PiecewiseRationalQuadraticCouplingTransform(
        (transform_net): ResidualNet(
          (initial_layer): Linear(in_features=15, out_features=50, bias=True)
          (blocks): ModuleList(
            (0-1): 2 x ResidualBlock(
              (context_layer): Linear(in_features=12, out_features=50, bias=True)
              (linear_layers): ModuleList(
                (0-1): 2 x Linear(in_features=50, out_features=50, bias=True)
              )
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (final_layer): Linear(in_features=50, out_features=87, bias=True)
        )
      )
      (1): LULinear()
      (2): PiecewiseRationalQuadraticCouplingTransform(
        (transform_net): ResidualNet(
          (initial_layer): Linear(in_features=15, out_features=50, bias=True)
          (blocks): ModuleList(
            (0-1): 2 x ResidualBlock(
              (context_laye

In [3]:
def calc_scores(xs, Cs, k=10):
    cal_samples = encoder.sample(k, xs).detach().cpu().numpy()
    cal_samples = cal_samples.reshape((cal_samples.shape[0], k, 2, 3))
    cal_tiled = np.transpose(np.tile(Cs.detach().cpu().numpy(), (k, 1, 1, 1)), (1, 0, 2, 3))
    cal_diff = cal_samples - cal_tiled
    cal_norms = np.linalg.norm(cal_diff, ord=2, axis=(2,3))
    return np.min(cal_norms, axis=-1)

In [4]:
# C in R^2x3
cal_sims = 1_000
cal_C_hats, cal_x = generate_data(cal_sims)
cal_C_hats = cal_C_hats.reshape((cal_sims, 2, 3))
cal_scores = calc_scores(cal_x, cal_C_hats)
q_hat = np.quantile(cal_scores, q = 0.95)

/home/yppatel/anaconda3/envs/chig/lib/python3.8/site-packages/nflows/transforms/lu.py:80: UserWarning: torch.triangular_solve is deprecated in favor of torch.linalg.solve_triangularand will be removed in a future PyTorch release.
torch.linalg.solve_triangular has its arguments reversed and does not return a copy of one of the inputs.
X = torch.triangular_solve(B, A).solution
should be replaced with
X = torch.linalg.solve_triangular(A, B). (Triggered internally at ../aten/src/ATen/native/BatchLinearAlgebra.cpp:2191.)
  outputs, _ = torch.triangular_solve(


In [5]:
k = 10
test_sims = 1
_, test_x = generate_data(test_sims)
test_C_hats = encoder.sample(k, test_x).detach().cpu().numpy()
test_C_hats = test_C_hats.reshape((test_C_hats.shape[0], k, 2, 3))

In [6]:
# Get system dynamics from side information
C_hat = test_C_hats[0][0]
A = C_hat[:,:2]
B = C_hat[:,2:]

# System problem data
Q = np.eye(A.shape[-1])
R = np.eye(B.shape[-1])
S0 = np.eye(A.shape[-1])

# useless for us, but just keep to use library code
Aa = np.zeros(A.shape)
Aa = Aa[:, :, np.newaxis]
Bb = np.zeros(B.shape)
Bb = Bb[:, :, np.newaxis]
a = np.zeros((1, 1))
b = np.zeros((1, 1))

In [7]:
opt_steps = 5_000

# Start with an initially stabilizing (feasible) controller;
# for this example the system is open-loop mean-square stable
A_star = np.copy(A)
B_star = np.copy(B)
K_star = np.zeros([B.shape[1], A.shape[1]])

PGO = PolicyGradientOptions(epsilon=(1e-2) * K_star.size,
                                eta=1e-3,
                                max_iters=1,
                                max_C_iters=500,
                                disp_stride=1,
                                keep_hist=True,
                                opt_method='proximal',
                                keep_opt='last',
                                step_direction='gradient',
                                stepsize_method='constant',
                                exact=True,
                                regularizer=Regularizer('vec1'),
                                regweight=0.0,
                                stop_crit='gradient',
                                fbest_repeat_max=0,
                                display_output=True,
                                display_inplace=True,
                                slow=False)

for opt_step in range(opt_steps):
    SS = LQRSysMult(A_star, B_star, a, Aa, b, Bb, Q, R, S0)
    SS.setK(K_star)

    # Run (regularized) policy gradient
    run_dynamics_gradient(SS, PGO, C_hat, q_hat) # find the C^* = [A^*, B^*] for Danskin's Theorem (with fixed controller K^(t))
    run_policy_gradient(SS, PGO)   # fix C^* from above and take a step to update K

    A_star = SS.A
    B_star = SS.B
    K_star = SS.K

    # Print the regularized optimal gains (from proximal gradient optimization)
    # and the unregularized optimal gains (from solving a Riccati equation)
    print('Optimized sparse gains')
    print(SS.K)
    print('Riccati gains')
    print(SS.Kare)

Iteration | Stop quant / threshold |  Curr obj |  Best obj | Norm of gain delta | Stepsize  | Sparsity
C iter: 0
2.022621703848887
C iter: 1
2.241581907378775
C iter: 2
2.0228086083781474
C iter: 3
2.2408717836604337
C iter: 4
2.022996307427155
C iter: 5
2.2401627457007356
C iter: 6
2.0231848087178115
C iter: 7
2.239454450384613
C iter: 8
2.0233741117444963
C iter: 9
2.238746570972343
C iter: 10
2.0235642020590268
C iter: 11
2.2380407987330218
C iter: 12
2.0237550874868653
C iter: 13
2.2373363239147395
C iter: 14
2.0239467684400205
C iter: 15
2.236633093044832
C iter: 16
2.024139252900555
C iter: 17
2.2359307966266737
C iter: 18
2.024332533782839
C iter: 19
2.235230024825275
C iter: 20
2.0245266038790675
C iter: 21
2.2345309284089208
C iter: 22
2.0247214798834374
C iter: 23
2.2338332432992107
C iter: 24
2.024917158873078
C iter: 25
2.2331370311370744
C iter: 26
2.0251136413408197
C iter: 27
2.2324422407748155
C iter: 28
2.0253109107047034
C iter: 29
2.231748342364394
C iter: 30
2.02550

KeyboardInterrupt: 